In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

# Import cifar 100 data and split
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()
x_val, y_val = x_train[35000:50000,:], y_train[35000:50000]
x_train, y_train = x_train[:35000,:], y_train[:35000]

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'vallidation samples')
print(x_test.shape[0], 'test samples')

x_train = x_train.astype('float32')/255
x_val = x_val.astype('float32')/255
x_test = x_test.astype('float32')/255

# Load pretrained model
base_model = DenseNet121(weights='imagenet', include_top=False)

x = base_model.output

x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.25)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.25)(x)

x = Dense(100)(x)

model = Model(inputs=base_model.input, outputs=x)

model.summary()


x_train shape: (35000, 32, 32, 3)
35000 train samples
15000 vallidation samples
10000 test samples
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
zero_padding2d_8 (ZeroPadding2D (None, None, None, 3 0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d_8[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]           

In [11]:
datagen = ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
    #zoom_range=0.3
    )
datagen.fit(x_train)

In [12]:
!mkdir ckpt2

mkdir: cannot create directory ‘ckpt2’: File exists


In [13]:
path1 = 'ckpt2/best.hdf5' 
Checkpoint = ModelCheckpoint(path1,monitor='val_loss',verbose= 1,save_best_only=True,save_weights_only=False,mode='min')
callback_list = [Checkpoint]

In [14]:
# Train only the top layers 
for layer in base_model.layers:
    layer.trainable = False

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
epochs = 50
learning_rate = 0.0005
decay_rate = learning_rate / epochs
opt = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=decay_rate, amsgrad=False)
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

# Train model
h = model.fit(x_train,y_train,
              epochs=15,
              validation_data=(x_val,y_val),
              verbose=1,
              batch_size=128,
              callbacks = callback_list)

Epoch 1/15
274/274 [==============================] - 15s 36ms/step - loss: 3.4497 - accuracy: 0.2171 - val_loss: 2.7506 - val_accuracy: 0.3191

Epoch 00001: val_loss improved from inf to 2.75063, saving model to ckpt2/best.hdf5
Epoch 2/15
274/274 [==============================] - 8s 30ms/step - loss: 2.6296 - accuracy: 0.3377 - val_loss: 2.5714 - val_accuracy: 0.3546

Epoch 00002: val_loss improved from 2.75063 to 2.57142, saving model to ckpt2/best.hdf5
Epoch 3/15
274/274 [==============================] - 8s 29ms/step - loss: 2.3486 - accuracy: 0.3919 - val_loss: 2.4776 - val_accuracy: 0.3656

Epoch 00003: val_loss improved from 2.57142 to 2.47760, saving model to ckpt2/best.hdf5
Epoch 4/15
274/274 [==============================] - 8s 31ms/step - loss: 2.1407 - accuracy: 0.4380 - val_loss: 2.4511 - val_accuracy: 0.3783

Epoch 00004: val_loss improved from 2.47760 to 2.45107, saving model to ckpt2/best.hdf5
Epoch 5/15
274/274 [==============================] - 8s 31ms/step - loss: 